# LangChain + Gemini RAG 간단 실습
- 문서를 불러와 쪼개고, 임베딩 → FAISS → Retriever → Gemini로 답변을 생성하는 최소 예제
- `requirements.txt`를 설치하고 실행하세요.

In [ ]:
# 필요 시 패키지 설치 (주석 해제해서 사용)
# !pip install -r ../requirements.txt

In [ ]:
import os, getpass, textwrap

if 'GEMINI_API_KEY' not in os.environ:
    os.environ['GEMINI_API_KEY'] = getpass.getpass('Enter GEMINI_API_KEY: ')
    print('GEMINI_API_KEY set from prompt')
else:
    print('GEMINI_API_KEY detected')


In [ ]:
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1) 로컬 문서 적재
with open('../data/sample.txt', 'r', encoding='utf-8') as f:
    text = f.read()
docs = [Document(page_content=text, metadata={'source': 'sample.txt'})]
print('Loaded docs:', len(docs))

# 2) Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = splitter.split_documents(docs)
print('Chunks:', len(splits))

# 3) 임베딩 + VectorStore (FAISS)
embeddings = GoogleGenerativeAIEmbeddings(model='models/text-embedding-004')
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# 4) 프롬프트 + 모델
prompt = ChatPromptTemplate.from_template(textwrap.dedent('''
    아래는 참고할 문서입니다. 답변 시 문서의 내용을 근거로 해주세요.
    ----------------
    {context}
    ----------------
    질문: {question}
    답변:'''
))
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash', temperature=0.3)

def format_docs(docs):
    return '

'.join(d.page_content for d in docs)

rag_chain = ({'context': retriever | format_docs, 'question': RunnablePassthrough()}
             | prompt
             | model
             | StrOutputParser())

answer = rag_chain.invoke('카페에서 파는 음료와 운영 시간을 알려줘')
print(answer)
